In [1]:
from argparse import Namespace
import lightning as L
import torch
from torchts.exp.lightning_exp import LightningExpMain

In [2]:
args = Namespace(
    model='Autoformer',
    data='ETTh1',
    root_path='../../../dataset/ETT-small',
    data_path='ETTh1.csv',
    features='MS',
    target='OT',
    freq='h',
    checkpoints='./informer_checkpoints',
    seq_len=96,
    label_len=48,
    pred_len=24,
    enc_in=7,
    dec_in=7,
    c_out=7,
    factor=5,
    d_model=512,
    n_heads=8,
    e_layers=2,
    d_layers=1,
    d_ff=2048,
    dropout=0.05,
    attn='prob',
    embed='timeF',
    activation='gelu',
    distil=True,
    output_attention=False,
    mix=True,
    padding=0,
    batch_size=32,
    learning_rate=1e-4,
    loss='mse',
    lradj='type1',
    use_amp=False,
    num_workers=0,
    itr=1,
    train_epochs=5,
    patience=5,
    des='exp',
    use_gpu=True if torch.cuda.is_available() else False,
    gpu=0,
    use_multi_gpu=False,
    devices='0,1,2,3',
    embed_type=0,
    train_only=False
)

In [3]:
from lightning.pytorch.callbacks import ModelCheckpoint
# Init ModelCheckpoint callback, monitoring 'val_loss'
checkpoint_callback = ModelCheckpoint(monitor="val_loss")

In [ ]:
model = LightningExpMain(args)
# Add your callback to the callbacks list
trainer = L.Trainer(accelerator='cpu', max_epochs=args.train_epochs, logger=True, callbacks=[checkpoint_callback],
                    enable_checkpointing=True,
                    )
trainer.fit(model)
trainer.test(model)
trainer.save_checkpoint('./informer_checkpoints')

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name      | Type    | Params
--------------------------------------
0 | model     | Model   | 11.3 M
1 | criterion | MSELoss | 0     
--------------------------------------
11.3 M    Trainable params
0         Non-trainable params
11.3 M    Total params
45.312    Total estimated model params size (MB)


train 8521
val 2857
test 2857


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

In [ ]:
for i, (path, _) in enumerate(trainer.checkpoint_callback.best_k_models.items()):
    m = model.load_from_checkpoint(path)
    m.transformer.save_pretrained(f'{i}th_best.pt')